**Connect google drive**

In [15]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Wed Feb 21 08:53:46 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [16]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
'Colab Notebooks'      Driveleech			    'My Drive'
"CrazyCousin's Vlog"   Muhammed_Zainuddin_Moosa_Resume.pdf   yolov3


**1) Clone the Darknet**



In [17]:
!git clone https://github.com/AlexeyAB/darknet

fatal: destination path 'darknet' already exists and is not an empty directory.


**2) Compile Darknet using Nvidia GPU**


In [18]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

[Errno 20] Not a directory: 'darknet'
/content/darknet
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:945:23: warning: variable ‘rgb’ set but not used []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-but-set-variable-Wunused-but-set-variable]8;;]
  945 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1443:14: warning: unused variab

**3) Configure Darknet network for training YOLO V3**

In [19]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [20]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [21]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [22]:
!echo "Koala" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

mkdir: cannot create directory ‘data/obj’: File exists


In [23]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2024-02-21 08:55:28--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74.1’

darknet53.conv.74.1 100%[===================>] 154.96M  43.6MB/s    in 3.8s    

2024-02-21 08:55:32 (40.5 MB/s) - ‘darknet53.conv.74.1’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [24]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/09d035c85d.jpg  
  inflating: data/obj/77ae6058e2.txt  
  inflating: data/obj/2aae24b29d.jpg  
  inflating: data/obj/5ba26c9060.jpg  
  inflating: data/obj/6a1cd7b301.jpg  
  inflating: data/obj/5a2d678016.jpg  
  inflating: data/obj/1a76c60c40.jpg  
  inflating: data/obj/34d06d0ded.jpg  
  inflating: data/obj/classes.txt    
  inflating: data/obj/7c6b58ad80.txt  
  inflating: data/obj/3d52baf740.txt  
  inflating: data/obj/4c78791147.txt  
  inflating: data/obj/0e8eb6b2e9.jpg  
  inflating: data/obj/1a660142ae.jpg  
  inflating: data/obj/9bf7dd7895.jpg  
  inflating: data/obj/4db506f55b.txt  
  inflating: data/obj/7ebb13cd63.jpg  
  inflating: data/obj/3b032f05b1.txt  
  inflating: data/obj/28a37c134f.txt  
  inflating: data/obj/33efc25c32.jpg  
  inflating: data/obj/063ba52395.txt  
  inflating: data/obj/57dd0f6d03.jpg  
  inflating: data/obj/0042f2effe.txt  
  inflating: data/obj/3c6d6206ec.jpg  
  inflating: data/obj/9ca7ba

In [25]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.771250 0.522000 0.379167 0.936000

['0', '0.771250', '0.522000', '0.379167', '0.936000']
0 data/obj/3b58222325.txt
0 0.771250 0.522000 0.379167 0.936000
0 0.616667 0.491023 0.419792 0.790008

['0', '0.616667', '0.491023', '0.419792', '0.790008']
1 data/obj/30fa42ded0.txt
0 0.616667 0.491023 0.419792 0.790008
0 0.417508 0.275148 0.464646 0.526627

['0', '0.417508', '0.275148', '0.464646', '0.526627']
2 data/obj/5fbc3e3756.txt
0 0.417508 0.275148 0.464646 0.526627
0 0.708754 0.633136 0.299663 0.449704

['0', '0.708754', '0.633136', '0.299663', '0.449704']
2 data/obj/5fbc3e3756.txt
0 0.708754 0.633136 0.299663 0.449704
0 0.327083 0.433796 0.520833 0.728704

['0', '0.327083', '0.433796', '0.520833', '0.728704']
3 data/obj/1a660142ae.txt
0 0.327083 0.433796 0.520833 0.728704
Koala

[]
0 0.288911 0.526770 0.424125 0.597582

['0', '0.288911', '0.526770', '0.424125', '0.597582']
5 data/obj/77ae6058e2.txt
0 0.288911 0.526770 0.424125 0.597582
0 0.723249 0.370466 0.331712 0.658031

['0', '0.

In [26]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/11fec2920e.jpg', 'data/obj/063ba52395.jpg', 'data/obj/9ca7babf11.jpg', 'data/obj/0eb924dd00.jpg', 'data/obj/27bcdbd57f.jpg', 'data/obj/77ae6058e2.jpg', 'data/obj/30be5547ce.jpg', 'data/obj/7c6b58ad80.jpg', 'data/obj/29d5fb0144.jpg', 'data/obj/4db506f55b.jpg', 'data/obj/0e8eb6b2e9.jpg', 'data/obj/2aae24b29d.jpg', 'data/obj/3f4b2076a8.jpg', 'data/obj/2e8fe24ad3.jpg', 'data/obj/30fa42ded0.jpg', 'data/obj/1f3883417c.jpg', 'data/obj/3c6d6206ec.jpg', 'data/obj/1a660142ae.jpg', 'data/obj/85a6803bf0.jpg', 'data/obj/9f8451e1c5.jpg', 'data/obj/9ccdbc321b.jpg', 'data/obj/51fb5f79ee.jpg', 'data/obj/4c78791147.jpg', 'data/obj/79dc8956bf.jpg', 'data/obj/3f1b884203.jpg', 'data/obj/6c55ea6cca.jpg', 'data/obj/1a9acbddfd.jpg', 'data/obj/9bf7dd7895.jpg', 'data/obj/8d2f165204.jpg', 'data/obj/53e6ef5129.jpg', 'data/obj/7d59076173.jpg', 'data/obj/28a37c134f.jpg', 'data/obj/1a76c60c40.jpg', 'data/obj/14be31fe67.jpg', 'data/obj/7d5fa491c6.jpg', 'data/obj/3d52baf740.jpg', 'data/obj/6ce9e3a265.jpg', 

In [27]:
#Create training.txt file
file = open("data/train.txt", "w")
file.write("\n".join(images_list))
file.close()

**6) Start the training**

In [28]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.349947), count: 5, class_loss = 1.404417, iou_loss = 2.585313, total_loss = 3.989730 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.026027, iou_loss = 0.000000, total_loss = 0.026027 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.024644, iou_loss = 0.000000, total_loss = 0.024644 
 total_bbox = 23932, rewritten_bbox = 0.000000 % 
326/4000: loss=0.9 hours left=5.2
 326: 0.886076, 0.916611 avg loss, 0.000011 rate, 7.156286 seconds, 20864 images, 5.197715 hours left
Loaded: 0.000112 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.307911), count: 4, class_loss = 1.146769, iou_loss = 1.955036, total_loss = 3.101805 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Regi